In [3]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import svm, metrics
from sklearn.linear_model import LogisticRegression as lr
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.preprocessing import Imputer, normalize, scale, MinMaxScaler, LabelEncoder, RobustScaler
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt
from sklearn.cross_validation import KFold, train_test_split as tts
from catboost import CatBoostClassifier as cbr
from sklearn.tree import DecisionTreeClassifier as dtc
import bokeh as bk
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from nltk import word_tokenize, sent_tokenize
from nltk.sentiment.util import mark_negation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
mms = MinMaxScaler()
rs = RobustScaler()
le = LabelEncoder()

C:\Users\visha\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
df = pd.read_csv('C:/Users/Visha/Desktop/train.csv', delimiter = ',', header = 0, index_col = None)

In [179]:
df.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [180]:
cleanup = {'Is_Response' : {'happy' : 1, 'not happy' : 0}}
df.replace(cleanup, inplace = 1)

In [181]:
df.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,0
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,0
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,1
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,0


In [5]:
df.Browser_Used.value_counts()

Firefox              7367
Edge                 7134
Google Chrome        4659
InternetExplorer     4588
Mozilla Firefox      4328
Mozilla              3092
Chrome               2470
IE                   2439
Internet Explorer    2103
Safari                390
Opera                 362
Name: Browser_Used, dtype: int64

In [183]:
df.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,0
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,0
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,1
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,0


In [186]:
print(df[df['Browser_Used'] == 'InternerExplorer'].Is_Response.describe())
print(df[df['Browser_Used'] != 'InternetExplorer'].Is_Response.describe())
df.info()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Is_Response, dtype: float64
count    34344.000000
mean         0.656709
std          0.474815
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Is_Response, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38932 entries, 0 to 38931
Data columns (total 5 columns):
User_ID         38932 non-null object
Description     38932 non-null object
Browser_Used    38932 non-null object
Device_Used     38932 non-null object
Is_Response     38932 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [187]:
df = pd.get_dummies(df, columns = ['Browser_Used', 'Device_Used'])
df.columns

Index(['User_ID', 'Description', 'Is_Response', 'Browser_Used_Chrome',
       'Browser_Used_Edge', 'Browser_Used_Firefox',
       'Browser_Used_Google Chrome', 'Browser_Used_IE',
       'Browser_Used_Internet Explorer', 'Browser_Used_InternetExplorer',
       'Browser_Used_Mozilla', 'Browser_Used_Mozilla Firefox',
       'Browser_Used_Opera', 'Browser_Used_Safari', 'Device_Used_Desktop',
       'Device_Used_Mobile', 'Device_Used_Tablet'],
      dtype='object')

In [188]:
df.Description.values[0]

"The room was kind of clean but had a VERY strong smell of dogs. Generally below average but ok for a overnight stay if you're not too fussy. Would consider staying again if the price was right. Breakfast was free and just about better than nothing."

In [133]:
sentiment_dict = {}
for line in open('C:/Users/Visha/Desktop/AFINN-111.txt'):
    word, score = line.split('\t')
    sentiment_dict[word] = int(score)
sentiment_dict

{'restoring': 1,
 'deferring': -1,
 'punishes': -2,
 'emergency': -2,
 'best': 3,
 'resentful': -2,
 'cute': 2,
 'darkest': -2,
 'struck': -1,
 'incapacitated': -2,
 'inspirational': 2,
 'missed': -2,
 'stubborn': -2,
 'lunatic': -3,
 'sincere': 2,
 'hurt': -2,
 'misinformation': -2,
 'matters': 1,
 'intimidating': -2,
 'glamourous': 3,
 'avoided': -1,
 'uneasy': -2,
 'blamed': -2,
 'destroyed': -3,
 'hahaha': 3,
 'irresponsible': 2,
 'await': -1,
 'bless': 2,
 'gullible': -2,
 'inflamed': -2,
 'choked': -2,
 'tumor': -2,
 'quaking': -2,
 'collapses': -2,
 'scandal': -3,
 'honoured': 2,
 'insignificant': -2,
 'upset': -2,
 'disabling': -1,
 'moan': -2,
 'annoyed': -2,
 'aggressions': -2,
 'interested': 2,
 'fails': -2,
 'screams': -2,
 'trembling': -2,
 'ass': -4,
 'supportive': 2,
 'favors': 2,
 'stressed': -2,
 'hid': -1,
 'audacious': 3,
 'depressed': -2,
 'starved': -2,
 'admits': -1,
 'monopolized': -2,
 'deniers': -2,
 'consent': 2,
 'bribe': -3,
 'abandon': -2,
 'rigorous': 3,
 

In [134]:
text = []
for i in df.Description.values:
    text.append(i)
text[0]

"The room was kind of clean but had a VERY strong smell of dogs. Generally below average but ok for a overnight stay if you're not too fussy. Would consider staying again if the price was right. Breakfast was free and just about better than nothing."

In [135]:
Pos_Score = []
Neg_Score = []
for t in text:
    pos = 0
    neg = 0
    for sentence in sent_tokenize(t):
        for word in word_tokenize(sentence):
            score = sentiment_dict.get(word, 0)
            if score > 0:
                pos += score
            if score < 0:
                neg += score
    Pos_Score.append(pos)
    Neg_Score.append(neg)

In [136]:
Positive = pd.Series(Pos_Score, dtype = 'int64', name = 'Positive')

In [137]:
Negative = pd.Series(Neg_Score, dtype = 'int64', name = 'Negative')

In [17]:
type(Negative)

pandas.core.series.Series

In [138]:
df.join([Positive, Negative])

,User_ID,Description,Is_Response,Device_Used_Desktop,Device_Used_Mobile,Device_Used_Tablet,Positive,Negative
0,id10326,The room was kind of clean but had a VERY stro...,not happy,0.0,1.0,0.0,9,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,not happy,0.0,1.0,0.0,4,-4
2,id10328,I booked this hotel through Hotwire at the low...,not happy,0.0,0.0,1.0,14,-10
3,id10329,Stayed here with husband and sons on the way t...,happy,1.0,0.0,0.0,27,0
4,id10330,My girlfriends and I stayed here to celebrate ...,not happy,0.0,0.0,1.0,21,-8
5,id10331,We had - rooms. One was very nice and clearly ...,happy,1.0,0.0,0.0,14,-3
6,id10332,My husband and I have stayed in this hotel a f...,not happy,0.0,0.0,1.0,8,-4
7,id10333,My wife & I stayed in this glorious city a whi...,happy,0.0,1.0,0.0,30,-1
8,id10334,My boyfriend and I stayed at the Fairmont on a...,happy,1.0,0.0,0.0,15,-2
9,id10335,"Wonderful staff, great location, but it was de...",not happy,0.0,0.0,1.0,13,-6


In [139]:
df.drop(['User_ID', 'Description'], inplace = 1, axis = 1)

In [140]:
df = df.join([Positive, Negative])

In [141]:
df.head()

,Is_Response,Device_Used_Desktop,Device_Used_Mobile,Device_Used_Tablet,Positive,Negative
0,not happy,0.0,1.0,0.0,9,0
1,not happy,0.0,1.0,0.0,4,-4
2,not happy,0.0,0.0,1.0,14,-10
3,happy,1.0,0.0,0.0,27,0
4,not happy,0.0,0.0,1.0,21,-8


In [142]:
y = df['Is_Response']
X = df.drop('Is_Response', axis = 1)

In [157]:
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.70, random_state = 7)

In [158]:
clf1 = GaussianNB()

In [159]:
clf1.fit(scale(X_train), y_train)

GaussianNB()

In [160]:
clf1.score(X_test, y_test)

0.78659230176494332

In [161]:
clf2 = svm.SVC(C = 0.5, kernel = 'linear', decision_function_shape = 'ovo', degree = 3, random_state = 7, cache_size = 1000)

In [162]:
clf2.fit(normalize(X_train, norm = 'l1', axis = 1), y_train)

SVC(C=0.5, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=7, shrinking=True,
  tol=0.001, verbose=False)

In [163]:
clf2.score(X_test, y_test)

0.78695923384581512

In [164]:
clf3 = knc(n_neighbors = 7, weights = 'uniform', algorithm = 'brute', p = 1)

In [165]:
clf3.fit(X_train, y_train)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=1,
           weights='uniform')

In [166]:
clf3.score(X_test, y_test)

0.7717682456977214

In [151]:
clf4 = rfc(n_estimators = 50, criterion = 'entropy', random_state = 7, max_features = 'sqrt')

In [152]:
clf4.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=7, verbose=0, warm_start=False)

In [153]:
clf4.score(X_test, y_test)

0.77851979598576304

In [53]:
clf5 = dtc(criterion = 'entropy', splitter = 'random', random_state = 7, max_features = 'sqrt')

In [54]:
clf5.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=7, splitter='random')

In [55]:
clf5.score(X_test, y_test)

0.76248486405166405

In [167]:
clf6 = lr(C = 50, solver = 'sag', max_iter = 999, random_state = 7)

In [168]:
clf6.fit(normalize(X_train, norm = 'l1', axis = 1), y_train)

LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=999, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=7, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [169]:
clf6.score(X_train, y_train)

0.78534121072009588

In [170]:
clf7 = svm.NuSVC(nu = 0.5, kernel = 'poly', decision_function_shape = 'ovo', degree = 2, random_state = 7, cache_size = 1000)

In [171]:
clf7.fit(normalize(X_train, norm = 'l1', axis = 1), y_train)

NuSVC(cache_size=1000, class_weight=None, coef0=0.0,
   decision_function_shape='ovo', degree=2, gamma='auto', kernel='poly',
   max_iter=-1, nu=0.5, probability=False, random_state=7, shrinking=True,
   tol=0.001, verbose=False)

In [172]:
clf7.score(X_test, y_test)

0.79238982864271823